## Task 1: Advanced Sequential Pipeline (Code Practice)

Design a **Complex Sequential Chain** that executes three steps:
1.  **Prompt 1:** Takes a user input of a `character_name` and requests the LLM to write a 100-word biography.
2.  **Chain 2:** The output (biography) from the first step is automatically passed as input to the second step.
3.  **Prompt 2:** Takes the biography and requests the LLM to identify and list three unique facts about the character mentioned in the biography.
4.  **Final Output:** The chain returns the list of three unique facts.

*Instructions:* Write the full LCEL syntax required, assuming you have `prompt_one`, `prompt_two`, `model`, and `parser` components defined.

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

In [2]:
load_dotenv()

model = ChatOpenAI(model="gpt-5-nano")

parser = StrOutputParser()

biography_prompt = PromptTemplate(
    template="Write a 100-120 word Biography of {person_name}.",
    input_variables=['person_name'],
)

facts_prompt = PromptTemplate(
    template="Give 3 unique facts about the person from its Biography.\n{biography}",
    input_variables=['biography'],
)

chain = biography_prompt | model | parser | facts_prompt | model | parser
print(chain.invoke({'person_name': 'Virat Kohli'}))

- Born on November 5, 1988, in Delhi.

- He is a right-handed batsman known for his modern batting technique.

- He served as captain of India's national team in all formats, leading the team to numerous victories and helping India reach the top of world rankings.


## Task 2: Parallel Chain for Content Audit (Code Practice)**
A user provides a long `text` (e.g., a corporate policy document). You need to process this text in parallel:
1.  **Branch A (Model 1):** Generate a short, formal `disclaimer` (using `prompt_disclaimer`).
2.  **Branch B (Model 2):** Generate a list of five key `keywords` (using `prompt_keywords`).
3.  **Merge:** Combine the disclaimer and the keywords list into a single structured summary document (using `prompt_merge`).

*Instructions:* Write the definition for the `parallel_chain`, the `merge_chain`, and the `final_chain` using `RunnableParallel` and LCEL. Assume two different models (`model_a`, `model_b`) and a single `parser` are available.

In [2]:
from langchain.schema.runnable import RunnableParallel

load_dotenv()

parser = StrOutputParser()

model = ChatOpenAI(model="gpt-5-nano")

text = """
## 📜 Corporate Policy Document: Acceptable Use of Information Technology Resources

**Document ID:** CORP-IT-AUP-001
**Version:** 3.0
**Effective Date:** January 1, 2026
**Review Date:** January 1, 2027
**Owner:** Chief Information Officer (CIO) / IT Governance Committee

---

### 1.0 Policy Statement and Purpose

It is the policy of **[Company Name]** ("the Company") to provide information technology (IT) resources to employees, contractors, and authorized third parties to facilitate business operations and enhance productivity. These resources are a valuable corporate asset. This policy establishes the acceptable standards for the use of all Company IT resources to ensure their security, integrity, and availability, and to ensure compliance with all relevant laws and contractual obligations.

### 2.0 Scope

This policy applies to **all individuals** who access or use the Company’s IT resources, including, but not limited to:
* Employees (full-time, part-time, temporary).
* Contractors, consultants, and interns.
* Authorized third parties.

IT Resources covered include all Company-owned or managed hardware, software, network systems, email, Internet access, communication platforms, and data, regardless of location.

### 3.0 Definitions

| Term | Definition |
| :--- | :--- |
| **IT Resources** | All computer hardware, software, network infrastructure, data, communication systems (email, VoIP, instant messaging), and physical access devices owned or leased by the Company. |
| **Acceptable Use** | Use of IT Resources that is consistent with the Company's business objectives, professional conduct standards, and all applicable laws and regulations. |
| **Unacceptable Use** | Any use of IT Resources that is illegal, unethical, harassing, detrimental to the Company's mission or reputation, or compromises system security or performance. |
| **Confidential Data** | Non-public information, including proprietary business plans, financial data, customer data, and employee PII (Personally Identifiable Information). |

### 4.0 General Usage Guidelines

#### 4.1 System Access and Security
* Users must protect their assigned User ID and password and are responsible for all activities conducted under their credentials.
* Sharing of passwords, credentials, or access tokens is strictly prohibited.
* All Company assets must be secured physically (e.g., locking workstations) and digitally (e.g., using strong passwords, enabling multi-factor authentication).
* Users must immediately report any known or suspected security breach, unauthorized access, or loss of an IT asset to the IT Security team.

#### 4.2 Data Handling and Classification
* All Company data must be stored on approved, secured Company IT Resources (e.g., corporate cloud drives, network shares).
* Users must adhere to the Company's Data Classification Policy and handle Confidential Data with the highest level of care, restricting access to authorized personnel only.
* Removal of Confidential Data from the corporate network (e.g., to personal devices or unsecured cloud services) is strictly forbidden without explicit, documented approval.

#### 4.3 Monitoring and Privacy
* Users acknowledge that the Company's IT Resources are intended for business use. **Users have no expectation of privacy** regarding their use of these resources, including emails, Internet browsing history, file usage, and communication logs.
* The Company reserves the right, and may exercise this right, to monitor, access, and disclose all information transmitted, received, or stored using its IT Resources without prior notice, as permitted by law and necessary for business, security, or legal compliance purposes.

### 5.0 Unacceptable Use

The following activities constitute unacceptable use and are strictly prohibited:
* **Illegal Activities:** Use of IT Resources for any illegal activity or in violation of local, state, federal, or international laws.
* **Harassment/Offensive Content:** Sending, retrieving, or displaying communications that are discriminatory, harassing, threatening, defamatory, or sexually explicit.
* **Unauthorized Software:** Installing, downloading, or using unauthorized software, shareware, or copyrighted materials in violation of licensing agreements.
* **Security Violations:** Circumventing security measures, attempting unauthorized access to systems or data, or performing network scanning/penetration testing without prior written IT approval.
* **Excessive Personal Use:** Excessive use of Internet or email for personal, non-work-related purposes that hinders productivity or consumes disproportionate bandwidth or storage.
* **Financial Gain:** Use of IT Resources for personal commercial activities or financial gain unrelated to the Company's business.

### 6.0 Enforcement and Violations

Violations of this policy will be subject to disciplinary action, up to and including immediate termination of employment or contract, and may result in civil and criminal penalties. The severity of the disciplinary action will depend on the nature and extent of the violation, as determined by the Legal and Human Resources departments.

---

### 7.0 Acknowledgment

I, the undersigned, acknowledge that I have read, understood, and agree to abide by the Corporate Policy Document: Acceptable Use of Information Technology Resources.

**Employee/User Signature:** __________________________________

**Print Name:** __________________________________

**Date:** __________________________________
"""

prompt_disclaimer = PromptTemplate(
    template="Write a short disclaimer message for the given policy\n{policy}",
    input_variables=['policy']
)

prompt_keywords = PromptTemplate(
    template="List down 5 important Keywords in the given Policy\n{policy}",
    input_variables=['policy']
)

prompt_merge =  PromptTemplate(
    template="Combine the following disclaimer and keywords to generate a summary of the given policy\nDisclaimer:\n{disclaimer}\nKeywords:\n{keywords}",
    input_variables=['disclaimer', 'keywords']
)

parallel_chain = RunnableParallel({
    'disclaimer': prompt_disclaimer | model | parser,
    'keywords': prompt_keywords | model | parser,
})

merge_chain = prompt_merge | model | parser

chain = parallel_chain | merge_chain

response = chain.invoke({'policy': text})
print(response)

Summary of policy (based on CORP-IT-AUP-001 and keywords):

- Purpose and scope: Defines the acceptable use, security, and confidentiality requirements for all IT Resources.
- Acknowledgement: By accessing or using IT Resources, you confirm you have read, understood, and agree to comply with the policy.
- Monitoring: The Company may monitor and audit all use of IT Resources without prior notice, as allowed by applicable law.
- Compliance and penalties: Violations may lead to disciplinary action up to termination and may involve civil or criminal penalties.
- Currency: The policy may be updated, and users should refer to the latest version for current requirements.
- Key areas covered: IT Resources, Acceptable Use, Confidential Data, Security, Monitoring.


## Task 3: Conditional Chain for Review Routing (Code/Concept Practice)**
You are building a review handling system using a **Conditional Chain**. The first step (Classification) determines if a user review's `urgency` is "High," "Medium," or "Low."

1.  If `urgency` is "High," trigger `chain_priority_support` (which sends an alert to a manager).
2.  If `urgency` is "Medium," trigger `chain_standard_response` (which sends an automated reply).
3.  If `urgency` is "Low," trigger `chain_archive` (which stores the review without immediate action).

*Instructions:*
*   Write the abstract structure of the `RunnableBranch` definition showing the conditions and the chains they trigger.
*   How would you handle the scenario where the LLM fails to classify the urgency (e.g., returns "Unclear")? What specialized Runnable component would be required for this default handling?